<a href="https://colab.research.google.com/github/balakrishna-24/video_ananlysis/blob/main/video_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries!

In [ ]:
!pip install opencv-python

!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.9 MB/s eta 0:00:00


In [ ]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install required dependencies
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 16960, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 16960 (delta 79), reused 95 (delta 45), pack-reused 16805 (from 1)
Receiving objects: 100% (16960/16960), 15.71 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (11612/11612), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.6/873.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
# Install necessary packages if not already available
!pip install torch torchvision

!pip install pytube whisper openai gtts
!pip install git+https://github.com/openai/whisper.git



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.1 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=ff4d52069ef2f068895a540dae2100b2f369f1fbcba02a660d4730bda09d5d72
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-luuqt9sa
  Running command git clone --filter=blo

In [ ]:
!pip install openai

In [ ]:
import torch
import cv2
import imutils
import time
import hashlib
from yt_dlp import YoutubeDL
from PIL import Image
import requests
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import os
import whisper
import openai  # Install with: pip install openai

# Set your OpenAI API key
openai.api_key = 'Your_api'  # Replace with your actual OpenAI API key

# Load models
# Load YOLOv5 model for object detection
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Load Places365 model for scene classification
model_places = models.resnet50(pretrained=False)
model_places.fc = torch.nn.Linear(model_places.fc.in_features, 365)
checkpoint = torch.hub.load_state_dict_from_url(
    'http://places2.csail.mit.edu/models_places365/resnet50_places365.pth.tar',
    map_location=torch.device('cpu')
)
state_dict = {k.replace('module.', ''): v for k, v in checkpoint['state_dict'].items()}
model_places.load_state_dict(state_dict)
model_places.eval()

# Define transformation for scene classification
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Load Places365 scene labels
LABELS_URL = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
response = requests.get(LABELS_URL)
categories = [line.strip().split(' ')[0][3:] for line in response.text.splitlines()]

# Load Whisper model for audio transcription
model_whisper = whisper.load_model("base")

# Function to download YouTube video
def download_youtube_video(url):
    url_hash = hashlib.md5(url.encode()).hexdigest()
    output_path = f'video_{url_hash}.mp4'
    ydl_opts = {
        'format': 'best',
        'outtmpl': output_path,
    }
    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"Downloaded video to {output_path}")
        return output_path
    except Exception as e:
        print(f"Error downloading video: {e}")
        return None

# Function to extract frames from the video
def get_frames(video_path, capture_interval=20):
    vs = cv2.VideoCapture(video_path)
    if not vs.isOpened():
        raise Exception(f'Unable to open file {video_path}')

    fps = vs.get(cv2.CAP_PROP_FPS)
    total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_count = 0
    frame_time = 0

    skip_frames = int(fps * capture_interval)

    while True:
        vs.set(cv2.CAP_PROP_POS_FRAMES, frame_count * skip_frames)
        success, frame = vs.read()
        if not success:
            break
        yield frame_count, frame_time, frame
        frame_count += 1
        frame_time += capture_interval

    vs.release()

# Object detection function using YOLOv5
def detect_objects(frame):
    results = model_yolo(frame)
    detected_objects = results.pandas().xyxy[0]['name'].tolist()
    return detected_objects

# Scene classification function using Places365
def classify_scene(frame):
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        logits = model_places(img_tensor)
        probs = torch.nn.functional.softmax(logits, dim=1)

    top5_probs, top5_cats = torch.topk(probs, 5)
    top_scene = categories[top5_cats[0][0]]
    return top_scene

# Function to extract and transcribe audio using Whisper
def extract_and_transcribe_audio(video_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    audio_file = 'audio.mp3'
    if os.path.exists(audio_file):
        print(f"Audio file downloaded: {audio_file}")
        result = model_whisper.transcribe(audio_file)
        transcription = result['text']
        print("Transcription:\n", transcription)
        return transcription
    else:
        print("Audio file not found!")
        return ""

# Function to process frames and collect visual data
def process_frames(video_path, capture_interval=20):
    visual_data = []
    for frame_count, frame_time, frame in get_frames(video_path, capture_interval):
        # Resize the frame for faster processing
        frame_resized = imutils.resize(frame, width=600)

        # Object detection
        print(f"Frame {frame_count}: Performing object detection")
        detected_objects = detect_objects(frame_resized)

        # Scene classification
        print(f"Frame {frame_count}: Performing scene classification")
        top_scene = classify_scene(frame_resized)

        # Collect visual information
        visual_info = {
            'frame_count': frame_count,
            'frame_time': frame_time,
            'scene': top_scene,
            'detected_objects': detected_objects,
        }
        visual_data.append(visual_info)
        print(f"Processed frame {frame_count} with scene '{top_scene}' and objects {detected_objects}")

    print(f'{len(visual_data)} frames processed with object detection and scene classification!')
    return visual_data

# Function to pass the combined data to the LLM
def analyze_with_llm(visual_data, audio_transcription):
    # Prepare the prompt
    prompt = "You are provided with the following visual and audio information extracted from a video.\n\n"

    # Add visual data to the prompt
    prompt += "Visual Information:\n"
    for info in visual_data:
        prompt += f"Frame {info['frame_count']} at {info['frame_time']}s:\n"
        prompt += f"- Scene: {info['scene']}\n"
        detected_objects = ', '.join(info['detected_objects']) if info['detected_objects'] else 'None'
        prompt += f"- Detected Objects: {detected_objects}\n\n"

    # Add audio transcription to the prompt
    prompt += "Audio Transcription:\n"
    prompt += audio_transcription.strip() + "\n\n"

    prompt += "Based on the above information, please provide a detailed analysis and explanation of the video's content, including any themes, events, or messages conveyed."

    # Prepare messages for the ChatCompletion API
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # Call the OpenAI ChatCompletion API
    response = openai.chat.completions.create(
        model="gpt-4",  # Or use "gpt-3.5-turbo" if you prefer
        messages=messages,
        max_tokens=500,   # Adjust as needed
        temperature=0.7   # Adjust for creativity
    )

    analysis = response.choices[0].message.content
    return analysis


if __name__ == "__main__":
    video_url = input("Please enter the YouTube video URL: ").strip()
    if video_url:
        print('Processing video_url:', video_url)

        # Step 1: Download the video
        video_path = download_youtube_video(video_url)
        if video_path:
            # Step 2: Process frames and collect visual data
            visual_data = process_frames(video_path, capture_interval=20)

            # Step 3: Extract and transcribe audio
            audio_transcription = extract_and_transcribe_audio(video_url)

            # Step 4: Combine visual and audio data and analyze with LLM
            analysis = analyze_with_llm(visual_data, audio_transcription)
            print("\nLLM Analysis and Explanation:\n")
            print(analysis)
        else:
            print("Failed to download video.")
    else:
        print("Invalid URL entered.")


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-9-20 Python-3.10.12 torch-2.4.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 44.6MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Downloading: "http://plac

Please enter the YouTube video URL: https://www.youtube.com/watch?v=Lx3gSNScFJA
Processing video_url: https://www.youtube.com/watch?v=Lx3gSNScFJA
[youtube] Extracting URL: https://www.youtube.com/watch?v=Lx3gSNScFJA
[youtube] Lx3gSNScFJA: Downloading webpage
[youtube] Lx3gSNScFJA: Downloading ios player API JSON
[youtube] Lx3gSNScFJA: Downloading web creator player API JSON
[youtube] Lx3gSNScFJA: Downloading player a9d81eca
[youtube] Lx3gSNScFJA: Downloading m3u8 information
[info] Lx3gSNScFJA: Downloading 1 format(s): 18
[download] Destination: video_fe7983658d3fd78d6c2a815640d24a1c.mp4
[download] 100% of   21.57MiB in 00:00:02 at 7.90MiB/s   
Downloaded video to video_fe7983658d3fd78d6c2a815640d24a1c.mp4
Frame 0: Performing object detection


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frame 0: Performing scene classification
Processed frame 0 with scene 'television_studio' and objects ['tie', 'person']
Frame 1: Performing object detection


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Frame 21: Performing scene classification
Processed frame 21 with scene 'television_studio' and objects ['person', 'person', 'tie', 'person', 'tie', 'tv', 'tie', 'potted plant']
Frame 22: Performing object detection


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frame 22: Performing scene classification
Processed frame 22 with scene 'television_studio' and objects ['person', 'tie', 'person', 'tie', 'person']
Frame 23: Performing object detection


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Frame 23: Performing scene classification
Processed frame 23 with scene 'television_studio' and objects ['person', 'tie', 'tv']
24 frames processed with object detection and scene classification!
[youtube] Extracting URL: https://www.youtube.com/watch?v=Lx3gSNScFJA
[youtube] Lx3gSNScFJA: Downloading webpage
[youtube] Lx3gSNScFJA: Downloading ios player API JSON
[youtube] Lx3gSNScFJA: Downloading web creator player API JSON
[youtube] Lx3gSNScFJA: Downloading m3u8 information
[info] Lx3gSNScFJA: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of    6.74MiB in 00:00:00 at 9.36MiB/s   
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)
Audio file downloaded: audio.mp3
Transcription:
  By the way, the Wall Street Journal is reporting when asked about whether Donald Trump was, in fact, ruling out a debate. And then we're playing games here. Spokeswoman simply said in an email, you heard the man. He's not debating. All rig